In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from utils.dataset import DataSet, LABELS
from models.own import OwnSingleModel
from keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score
from keras.utils import plot_model

from env import *

%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs_sw', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

VECTOR_DIM = 300
EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem_%d.pkl' % VECTOR_DIM)

MODEL_FILE = os.path.join(SAVED_MODELS_PATH, 'own', 'single.h5')
RESULT_CSV = os.path.join(RESULTS_PATH, 'own_single.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

seq = 'words_seq'
train_x = np.array(list(train_with_seq[seq]))
val_x = np.array(list(val_with_seq[seq]))
test_x = np.array(list(test_with_seq[seq]))

y_cols = LABELS
val_ys = []
train_outputs = []
val_outputs = []
for col in y_cols:
    train_y = train_with_seq[col] + 2
    val_y = val_with_seq[col] + 2
    val_ys.append(val_y)
    train_y_onehot = to_categorical(train_y)
    val_y_onehot = to_categorical(val_y)
    train_outputs.append(train_y_onehot)
    val_outputs.append(val_y_onehot)

In [6]:
model = OwnSingleModel(max_len = train_x.shape[1], embedding = embedding)
model._model.summary()
plot_model(model._model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1123)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1123, 300)    15000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 1123, 300)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
cu_dnngru_1 (CuDNNGRU)          (None, 1123, 300)    541800      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
cu_dnngru_

In [7]:
model.fit(train_x, train_outputs, 
          validation_data = (val_x, val_outputs),
          model_file = MODEL_FILE
         )

Train on 105000 samples, validate on 15000 samples
Epoch 1/300
105000/105000 [==============================] - 1447s 14ms/step - loss: 12.9082 - dense_1_loss: 0.4764 - dense_2_loss: 0.5348 - dense_3_loss: 0.5852 - dense_4_loss: 0.4704 - dense_5_loss: 0.7286 - dense_6_loss: 0.2684 - dense_7_loss: 0.4943 - dense_8_loss: 0.8353 - dense_9_loss: 0.5247 - dense_10_loss: 0.6607 - dense_11_loss: 0.7634 - dense_12_loss: 0.6936 - dense_13_loss: 0.8721 - dense_14_loss: 0.7714 - dense_15_loss: 0.9702 - dense_16_loss: 0.7871 - dense_17_loss: 0.7755 - dense_18_loss: 0.5498 - dense_19_loss: 0.6522 - dense_20_loss: 0.4942 - val_loss: 8.0671 - val_dense_1_loss: 0.1987 - val_dense_2_loss: 0.3252 - val_dense_3_loss: 0.2803 - val_dense_4_loss: 0.3051 - val_dense_5_loss: 0.4312 - val_dense_6_loss: 0.1072 - val_dense_7_loss: 0.2450 - val_dense_8_loss: 0.5754 - val_dense_9_loss: 0.3311 - val_dense_10_loss: 0.4433 - val_dense_11_loss: 0.4384 - val_dense_12_loss: 0.4204 - val_dense_13_loss: 0.5442 - val_dense

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00001: _f1_monitor improved from -inf to 0.60817, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/own/single.h5
Epoch 2/300
105000/105000 [==============================] - 1450s 14ms/step - loss: 8.3144 - dense_1_loss: 0.2159 - dense_2_loss: 0.3246 - dense_3_loss: 0.3006 - dense_4_loss: 0.3069 - dense_5_loss: 0.4463 - dense_6_loss: 0.1018 - dense_7_loss: 0.2588 - dense_8_loss: 0.5820 - dense_9_loss: 0.3520 - dense_10_loss: 0.4675 - dense_11_loss: 0.4630 - dense_12_loss: 0.4371 - dense_13_loss: 0.5575 - dense_14_loss: 0.4610 - dense_15_loss: 0.6632 - dense_16_loss: 0.5647 - dense_17_loss: 0.5769 - dense_18_loss: 0.3379 - dense_19_loss: 0.5136 - dense_20_loss: 0.3832 - val_loss: 7.5465 - val_dense_1_loss: 0.1747 - val_dense_2_loss: 0.2955 - val_dense_3_loss: 0.2581 - val_dense_4_loss: 0.2711 - val_dense_5_loss: 0.4245 - val_dense_6_loss: 0.0823 - val_dense_7_loss: 0.2191 - val_dense_8_loss: 0.5433 - val_dense_9_loss: 0.3216 - val_dense_10_loss: 0.

105000/105000 [==============================] - 1452s 14ms/step - loss: 6.1666 - dense_1_loss: 0.1497 - dense_2_loss: 0.2384 - dense_3_loss: 0.2112 - dense_4_loss: 0.2372 - dense_5_loss: 0.3160 - dense_6_loss: 0.0708 - dense_7_loss: 0.1769 - dense_8_loss: 0.4425 - dense_9_loss: 0.2878 - dense_10_loss: 0.3764 - dense_11_loss: 0.3395 - dense_12_loss: 0.3198 - dense_13_loss: 0.4000 - dense_14_loss: 0.3359 - dense_15_loss: 0.4904 - dense_16_loss: 0.4094 - dense_17_loss: 0.4563 - dense_18_loss: 0.2699 - dense_19_loss: 0.3561 - dense_20_loss: 0.2825 - val_loss: 7.3892 - val_dense_1_loss: 0.1812 - val_dense_2_loss: 0.3078 - val_dense_3_loss: 0.2706 - val_dense_4_loss: 0.2525 - val_dense_5_loss: 0.3935 - val_dense_6_loss: 0.0738 - val_dense_7_loss: 0.2049 - val_dense_8_loss: 0.5252 - val_dense_9_loss: 0.3137 - val_dense_10_loss: 0.4058 - val_dense_11_loss: 0.3981 - val_dense_12_loss: 0.3718 - val_dense_13_loss: 0.4615 - val_dense_14_loss: 0.3697 - val_dense_15_loss: 0.5856 - val_dense_16_loss

In [8]:
model._model.load_weights(MODEL_FILE)
val_probs = model.predict(val_x)
test_probs = model.predict(test_x)

val_preds = list(map(lambda x: np.argmax(x, axis = -1), val_probs))
# val_preds = np.argmax(val_probs, axis = 1)
# test_preds = list(map(lambda x: np.argmax(x, axis = -1), test_probs))

In [9]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_ys, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.6425371792941685
The 1th f1: 0.5141235370954264
The 2th f1: 0.695493744582423
The 3th f1: 0.640088365444609
The 4th f1: 0.7837327279181838
The 5th f1: 0.7172420553778831
The 6th f1: 0.7369990377189853
The 7th f1: 0.7621560943307086
The 8th f1: 0.7063024045200141
The 9th f1: 0.6660590777499031
The 10th f1: 0.6903950824341004
The 11th f1: 0.7494882070247237
The 12th f1: 0.7371395409067902
The 13th f1: 0.7298458633368893
The 14th f1: 0.7012713630749309
The 15th f1: 0.7054886126198998
The 16th f1: 0.55434074801027
The 17th f1: 0.7307448277835289
The 18th f1: 0.5632610454578739
The 19th f1: 0.7135036753532185
The average f1 of val is 0.6870106595017266


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
